In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle


In [2]:
base = '/home/ana/Downloads/'

In [3]:
df = pd.read_pickle(base + 'df_clean')

In [12]:
df.head(50)

,Id_Historico_Prestamo,Id_Usuario,Id_Tag_Bicicleta,Fecha_Prestamo,Fecha_Devolucion,Id_Aparcamiento_Origen,Posicion_Origen,Id_Aparcamiento_Destino,Posicion_Destino,operario,Num_Bici_Hist,Duration,Year_Prestamo,Year_Devolucion
0,6501462,18701,1164,2017-06-21 14:22:11.000,2017-06-21 14:38:29.000,8,1,43.0,1.0,161,929.0,978.0,2017,2017
1,6501318,41811,1089,2017-06-21 14:07:18.000,2017-06-21 14:39:06.000,56,6,54.0,NaN,251,1612.0,1908.0,2017,2017
2,6501489,96620,1110,2017-06-21 14:25:11.000,2017-06-21 14:39:07.000,36,25,9.0,13.0,273,1679.0,836.0,2017,2017
3,6501001,16055,1465,2017-06-21 13:35:52.000,2017-06-21 14:39:22.000,8,1,9.0,6.0,273,1076.0,3810.0,2017,2017
4,6501421,6431,996,2017-06-21 14:17:56.000,2017-06-21 14:39:26.000,56,14,54.0,NaN,251,1176.0,1290.0,2017,2017
5,6501591,57428,1400,2017-06-21 14:35:33.000,2017-06-21 14:39:33.000,22,8,47.0,9.0,273,550.0,240.0,2017,2017
6,6501386,45161,270,2017-06-21 14:15:10.000,2017-06-21 14:39:39.000,56,12,54.0,NaN,251,759.0,1469.0,2017,2017
7,6501477,34373,1158,2017-06-21 14:23:32.000,2017-06-21 14:39:46.000,19,33,11.0,22.0,273,860.0,974.0,2017,2017
8,6500993,25026,380,2017-06-21 13:34:31.000,2017-06-21 14:39:53.000,21,1,54.0,NaN,251,878.0,3922.0,2017,2017
9,6500917,34894,778,2017-06-21 13:27:28.000,2017-06-21 14:40:02.000,47,19,54.0,NaN,251,601.0,4354.0,2017,2017


Verificar quando as estações automáticas cheias. -> ver para cada estação qual o nº máximo de posições diferentes e depois tentar perceber qnd é que elas estão cheias (elas estão cheias se num dado intervalo de tempo todas as positions estão ocupadas) ---> taxa de ocupação

In [5]:
# Get the maximum number of positions in a station
# Id_Aparcamiento_Origen -> 128
# Id_Aparcamiento_Destino -> 142 diferentes

## APAGAR!!

def get_max_available_positions_per_station(_df):
    df = _df.copy()
    # Pandas drop_duplicates() function removes duplicate rows from the DataFrame
    df_origem = df[['Id_Aparcamiento_Origen', 'Posicion_Origen']] \
                    .drop_duplicates() \
                    .rename(columns={"Id_Aparcamiento_Origen": "Id_Aparcamiento", "Posicion_Origen": "Position"})
    df_destino = df[['Id_Aparcamiento_Destino', 'Posicion_Destino']] \
                    .drop_duplicates() \
                    .rename(columns={"Id_Aparcamiento_Destino": "Id_Aparcamiento", "Posicion_Destino": "Position"})
    
    df = pd.concat([df_origem, df_destino]).drop_duplicates()
    df_apps = df.groupby('Id_Aparcamiento')['Position'].nunique().to_frame()
    df_apps['Max Position'] = df.groupby('Id_Aparcamiento').max()
    # 0 se é manual, 1 se é automática
    df_apps['Type'] = [ 0 if x is 0 else 1 for x in df_apps['Position']]
    
    return df_apps
    
df_pos_per_station = get_max_available_positions_per_station(df)
df_pos_per_station.head()

,Position,Max Position,Type
Id_Aparcamiento,,,
0.0,0,NaN,0
1.0,1,1.0,1
2.0,1,1.0,1
3.0,40,40.0,1
4.0,1,1.0,1


In [14]:
df_pos_per_station

,Position,Max Position,Type
Id_Aparcamiento,,,
0.0,0,NaN,0
1.0,1,1.0,1
2.0,1,1.0,1
3.0,40,40.0,1
4.0,1,1.0,1
...,...,...,...
113.0,19,20.0,1
114.0,19,19.0,1
115.0,28,28.0,1


Há posições que nunca são ocupadas!!!!

In [6]:
def occupation_tax(nPos, maxPos):
    return (nPos*100)/maxPos

In [11]:
# obter  a posição inicial da bicicleta de forma a saber quantas bicicletas tem cada estação inicialmente



def get_bike_location(_df, station):

    df = _df.copy()
    df = df[['Id_Tag_Bicicleta', 'Fecha_Prestamo', 'Fecha_Devolucion', 'Id_Aparcamiento_Origen', \
             'Posicion_Origen', 'Id_Aparcamiento_Destino', 'Posicion_Destino']]
    
    df = df[(df['Id_Aparcamiento_Origen']==station) | (df['Id_Aparcamiento_Destino']==station)]
    #df_ts = df_ts.set_index(['Fecha_Prestamo', 'month'])

    return df

# eles podem ter bicicletas na estação q nós não sabemos porque nunca são emprestadas!!!!!!!!!!!!!!!!!!!!!!!!!
get_bikes_per_station(df, 8)

,Id_Tag_Bicicleta,Fecha_Prestamo,Fecha_Devolucion,Id_Aparcamiento_Origen,Posicion_Origen,Id_Aparcamiento_Destino,Posicion_Destino
0,1164,2017-06-21 14:22:11.000,2017-06-21 14:38:29.000,8,1,43.0,1.0
3,1465,2017-06-21 13:35:52.000,2017-06-21 14:39:22.000,8,1,9.0,6.0
36,1028,2017-06-21 14:32:59.000,2017-06-21 14:45:49.000,8,1,22.0,8.0
37,1224,2017-06-21 14:32:32.000,2017-06-21 14:45:52.000,8,1,22.0,6.0
38,1092,2017-06-21 14:39:27.000,2017-06-21 14:46:08.000,8,1,10.0,1.0
...,...,...,...,...,...,...,...
10387726,2329,2019-12-31 13:27:06.000,2019-12-31 13:41:43.000,8,1,48.0,8.0
10387730,2011,2019-12-31 13:20:37.000,2019-12-31 14:12:46.000,8,1,48.0,13.0
10387731,1836,2019-12-31 14:01:59.000,2019-12-31 14:13:35.000,8,1,53.0,16.0
10387734,2751,2019-12-31 13:58:21.000,2019-12-31 14:59:13.000,10,1,8.0,1.0


In [ ]:
df.columns

Ver a taxa de ocupação a cada hora

Ver a capacidade de armazenamento das estações automáticas. Quais são as estações automáticas com maior capacidade de armazenamento? Quais as com menor?

Quais são as horas/dias da semana/meses mais comuns para usar uma bicicleta?

In [ ]:
df.Fecha_Prestamo=pd.to_datetime(df.Fecha_Prestamo)
df.Fecha_Devolucion=pd.to_datetime(df.Fecha_Devolucion)
df['year_prestamo'] = df.Fecha_Prestamo.dt.year
df['month_prestamo'] = df.Fecha_Prestamo.dt.month
df['day_prestamo'] = df.Fecha_Prestamo.dt.day
df['hour_prestamo'] = df.Fecha_Prestamo.dt.hour
df

In [ ]:
def plot_prestamo_over_time(df, time_unit):
    dfYear=df.groupby(time_unit+'_prestamo').size().reset_index(name='count')
    print(dfYear)
    ax = dfYear.plot(x=time_unit+"_prestamo")
    ax.locator_params(integer=True)

In [ ]:
plot_prestamo_over_time(df, "year")

In [ ]:
plot_prestamo_over_time(df, "month")

In [ ]:
plot_prestamo_over_time(df, "day")

In [ ]:
plot_prestamo_over_time(df, "hour")

In [ ]:
Quais é que são os tempos médios de empréstimo?


In [ ]:
df['duration_hour'] = (df.Fecha_Devolucion-df.Fecha_Prestamo).dt.total_seconds().div(60*60)
df

In [ ]:
dfDuration=df.groupby('duration_hour').size().reset_index(name='count')
ax=dfDuration.plot(x="duration_hour", y="count", figsize=(16,9))
ax.set_xscale('log')

In [ ]:
Quais as origens/destinos mais comuns?

In [ ]:
dfOrigin=df.groupby('Id_Aparcamiento_Origen').size().reset_index(name='count')
dfOrigin.plot(x="Id_Aparcamiento_Origen", figsize=(16,9))

In [ ]:
dfDestination=df.groupby('Id_Aparcamiento_Destino').size().reset_index(name='count')
dfDestination.plot(x="Id_Aparcamiento_Destino", figsize=(16,9))

In [ ]:
Quais os trajetos mais comuns?

In [ ]:
dfTranjects=df.groupby(['Id_Aparcamiento_Origen', 'Id_Aparcamiento_Destino']).size().reset_index(name='count')
dfTranjects

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(16, 9))
ax = Axes3D(fig)

ax.plot_trisurf(dfTranjects["Id_Aparcamiento_Origen"], dfTranjects["Id_Aparcamiento_Destino"], dfTranjects["count"], cmap=cm.jet, linewidth=0.2)
ax.set_zlabel('Count')
ax.set_xlabel('Origin')
ax.set_ylabel('Destination')

plt.show()

In [ ]:
Cadeias de markov ou função matemática: qual é a probabilidade de uma bicicleta ser transportada para uma estação X sabendo que estava na estação Y